In [1]:
import pandas as pd
import os
import numpy as np
import datetime
from plotnine import *
import matplotlib.pyplot as plt
from model_diagnostics import model_diagnostics, skf_preds, model_diagnostics_skf, summarise_continuous_feature

#pd.set_option("display.max_rows", 20)

In [31]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml(
    "titanic", version=1, as_frame=True, return_X_y=True
)

drop_cols = ['boat', 'body', 'home.dest']

X.drop(columns = drop_cols, inplace = True)

#shuffle data, split into train and test (891 rows in train)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S
...,...,...,...,...,...,...,...,...,...,...
1304,3.0,"Zabour, Miss. Hileni",female,14.5000,1.0,0.0,2665,14.4542,None,C
1305,3.0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,None,C
1306,3.0,"Zakarian, Mr. Mapriededer",male,26.5000,0.0,0.0,2656,7.2250,None,C
1307,3.0,"Zakarian, Mr. Ortin",male,27.0000,0.0,0.0,2670,7.2250,None,C


In [32]:
filename = os.getcwd() + "/train.csv"
data = pd.read_csv(filename)

In [33]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Feature Engineering

## Construct deck feature

In [4]:
(
    data
    .assign(Deck = data['Cabin'].str[0].fillna('M'),
            CabinNo = data['Cabin'].str.split('(\d+)', expand = True)[1].fillna(0).astype(int))
    .head(1)
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,CabinNo
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,M,0


In [5]:
data['Deck'] = data['Cabin'].str[0]
#data.loc[data['Cabin'].isnull(), 'Deck'] = 'M'
data['Deck'] = data['Deck'].fillna('M')
data['CabinNo'] = data['Cabin'].str.split('(\d+)', expand = True)[1].fillna(0).astype(int)

In [6]:
survival_deck = (
    data
    .groupby('Deck')
    .agg(n = ('SibSp', 'count'),
         pct_survived = ('Survived', 'mean'))
    .reset_index()
)

survival_deck

,Deck,n,pct_survived
0,A,15,0.466667
1,B,47,0.744681
2,C,59,0.593220
3,D,33,0.757576
4,E,32,0.750000
5,F,13,0.615385
6,G,4,0.500000
7,M,687,0.299854
8,T,1,0.000000


In [7]:
#df_all['Deck'] = df_all['Deck'].replace(['A', 'B', 'C'], 'ABC')
data['Deck'] = data['Deck'].replace(['A', 'T'], 'A')
data['Deck'] = data['Deck'].replace(['F', 'G'], 'FG')
data['Deck'].value_counts()

M     687
C      59
B      47
D      33
E      32
FG     17
A      16
Name: Deck, dtype: int64

## Extract Title feature

In [8]:
data['Title'] = data['Name'].str.split(",", expand = True)[1].str.split('.', expand = True)[0].str.strip()
data['Title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [9]:
survival_title = (
    data
    .groupby('Title')
    .agg(n = ('Name', 'count'),
         avg_survival = ('Survived', 'mean'))
    .reset_index()
)

survival_title

,Title,n,avg_survival
0,Capt,1,0.000000
1,Col,2,0.500000
2,Don,1,0.000000
3,Dr,7,0.428571
4,Jonkheer,1,0.000000
5,Lady,1,1.000000
6,Major,2,0.500000
7,Master,40,0.575000
8,Miss,182,0.697802
9,Mlle,2,1.000000


In [10]:
data.query('Name.str.contains("Mme")', engine = "python")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,CabinNo,Title
369,370,1,1,"Aubart, Mme. Leontine Pauline",female,24.0,0,0,PC 17477,69.3,B35,C,B,35,Mme


In [11]:
data['Title'] = data['Title'].replace(['Ms'], 'Miss')

noble_list = ['Dr', 'Rev', 'Mlle', 'Major', 'Col', 'the Countess', 'Capt', 'Sir', 'Lady', 'Mme', 'Don', 'Jonkheer']
#noble_list = ['Mlle', 'the Countess', 'Sir', 'Lady', 'Mme', 'Don', 'Jonkheer']
#military_clergy_list = ['Rev', 'Major', 'Col', 'Capt']

data['Title'] = data['Title'].replace(noble_list, 'Noble')
#data['Title'] = data['Title'].replace(military_clergy_list, 'Military_Clergy')
#data['Title'] = data['Title'].replace('Dr', 'Mr')

data['Title'].value_counts()

Mr        517
Miss      183
Mrs       125
Master     40
Noble      26
Name: Title, dtype: int64

In [12]:
survival_title = (
    data
    .groupby('Title')
    .agg(n = ('Name', 'count'),
         avg_survival = ('Survived', 'mean'))
    .reset_index()
)

survival_title

,Title,n,avg_survival
0,Master,40,0.575000
1,Miss,183,0.699454
2,Mr,517,0.156673
3,Mrs,125,0.792000
4,Noble,26,0.423077


In [13]:
data['Married'] = 0
data.loc[data['Title'] == 'Mrs', 'Married'] = 1

## One hot encode categoricals

In [14]:
cat_columns = ['Sex', 'Pclass', 'Embarked', 'Title', 'Deck']
pd.concat([data, pd.get_dummies(data, columns = cat_columns, dummy_na = True, drop_first = True)], axis = 1).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Title_Mrs,Title_Noble,Title_nan,Deck_B,Deck_C,Deck_D,Deck_E,Deck_FG,Deck_M,Deck_nan
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,0,0,0,0,0,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,1,0,0,0,1,0,0,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,0,0,0,0,0,0,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,1,0,0,0,1,0,0,0,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,0,0,0,0,0,0,0,1,0


# Test whether you can get feature names out of sklearn pipeline dummies

In [72]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
import xgboost as xgb

target = 'Survived'
numeric_cols = ['SibSp', 'Parch', 'Fare', 'Age']
categorical_cols = ['Pclass', 'Sex', 'Embarked']

X, y = data[numeric_cols + categorical_cols].copy(), np.asarray(data[target], dtype = 'int8')
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 20230301)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 20230118)


ct_encode = ColumnTransformer([
#    ('scaler', StandardScaler(), numeric_cols), #if just this line, the pipeline will only return the four numeric columns, scaled
    ('encoder', OneHotEncoder(handle_unknown='ignore'), categorical_cols), #this line adds dummies for class (3 values), sex (2 values) and embarked (4 values)
    ('pass_through_numerics', 'passthrough', numeric_cols)
])

ct_disc = ColumnTransformer(
    transformers = [('disc_age', KBinsDiscretizer(n_bins = 10, encode = 'ordinal'), [-1]),
                    ('disc_fare', KBinsDiscretizer(n_bins = 15, encode = 'ordinal'), [-2]),
                    #('disc_cabno', StandardScaler(), [-3])
                    ],
    remainder = 'passthrough'
)

ct_scale = ColumnTransformer(
    transformers = [('scale', StandardScaler(), [-1, -2])],
    remainder = 'passthrough'
)

xgb_model = xgb.XGBClassifier(eval_metric = 'logloss')

pipe = Pipeline([
    ('encode_cats', ct_encode),
    ('imputer', SimpleImputer()),
    ('scale', ct_scale),
    ('clf', xgb_model),
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('encode_cats',
                 ColumnTransformer(transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Pclass', 'Sex',
                                                   'Embarked']),
                                                 ('pass_through_numerics',
                                                  'passthrough',
                                                  ['SibSp', 'Parch', 'Fare',
                                                   'Age'])])),
                ('imputer', SimpleImputer()),
                ('scale',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  [-1, -2])])),
                ('c...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_p

In [56]:
pipe.feature_names_in_

array(['SibSp', 'Parch', 'Fare', 'Age', 'Pclass', 'Sex', 'Embarked'],
      dtype=object)

In [69]:
pipe.named_steps['encode_cats'].get_feature_names_out()

array(['encoder__Pclass_1', 'encoder__Pclass_2', 'encoder__Pclass_3',
       'encoder__Sex_female', 'encoder__Sex_male', 'encoder__Embarked_C',
       'encoder__Embarked_Q', 'encoder__Embarked_S',
       'encoder__Embarked_nan', 'pass_through_numerics__SibSp',
       'pass_through_numerics__Parch', 'pass_through_numerics__Fare',
       'pass_through_numerics__Age'], dtype=object)

In [73]:
pipe.named_steps['imputer'].get_feature_names_out()

AttributeError: 'SimpleImputer' object has no attribute 'get_feature_names_out'

In [68]:
pipe.named_steps['scale'].get_feature_names_out()

array(['scale__x12', 'scale__x11', 'remainder__x0', 'remainder__x1',
       'remainder__x2', 'remainder__x3', 'remainder__x4', 'remainder__x5',
       'remainder__x6', 'remainder__x7', 'remainder__x8', 'remainder__x9',
       'remainder__x10'], dtype=object)

# Appendix: Annoying times working with sklearn OneHotEncoder

In [15]:
# from sklearn.preprocessing import OneHotEncoder

# enc = OneHotEncoder()
# X = data[['Sex', 'Pclass', 'Embarked']].copy()
# enc.fit_transform(X).toarray()

In [16]:
# enc.get_feature_names_out()

In [17]:
# cat_features = ['Pclass', 'Sex', 'Embarked']
# encoded_features = []
# dfs = [data]

# for df in dfs:
#     for feature in cat_features:
#         encoded_feat = OneHotEncoder().fit_transform(df[feature].values.reshape(-1, 1)).toarray()
#         n = df[feature].nunique()
#         cols = ['{}_{}'.format(feature, n) for n in range(1, n + 1)]
#         encoded_df = pd.DataFrame(encoded_feat, columns=cols)
#         encoded_df.index = df.index
#         encoded_features.append(encoded_df)


In [18]:
# cols 

In [19]:
# encoded_feat